In [ ]:
# carregar dataset rudementar
# ==============================================================
# CARREGAR DATASET RUDIMENTAR — Coleta de POIs do OSM
# Angola (Pontos Turísticos com coordenadas conhecidas)
# ==============================================================

import osmnx as ox
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point
from pathlib import Path
import time

# === Diretório de saída ===
DATA_DIR = Path("../data/geoespacial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

# === Locais turísticos (com coordenadas conhecidas) ===
locais_turisticos = [
    # ============================================================
    #  CUNENE
    # ============================================================
    {"nome": "Quedas do Ruacaná", "lat": -17.3761, "lon": 14.2258, "provincia": "Cunene"},
    {"nome": "Parque Nacional da Mupa", "lat": -16.4167, "lon": 15.5833, "provincia": "Cunene"},
    {"nome": "Memorial do Rei Mandume ya Ndemufayo", "lat": -17.2619, "lon": 15.6542, "provincia": "Cunene"},

    # ============================================================
    #  CUANZA NORTE
    # ============================================================
    {"nome": "Jardim Botânico de Kilombo", "lat": -9.2981, "lon": 14.9125, "provincia": "Cuanza Norte"},
    {"nome": "Fortaleza de Massangano", "lat": -9.6386, "lon": 14.2372, "provincia": "Cuanza Norte"},
    {"nome": "Barragem de Cambambe", "lat": -9.7550, "lon": 14.4847, "provincia": "Cuanza Norte"},

    # ============================================================
    #  CUANZA SUL
    # ============================================================
    {"nome": "Cachoeiras do Binga", "lat": -11.3541, "lon": 14.1678, "provincia": "Cuanza Sul"},
    {"nome": "Grutas de Sassa", "lat": -11.1333, "lon": 14.4833, "provincia": "Cuanza Sul"},
    {"nome": "Porto Amboim", "lat": -10.7250, "lon": 13.7583, "provincia": "Cuanza Sul"},
    {"nome": "Praia da Sumbe", "lat": -11.2053, "lon": 13.8436, "provincia": "Cuanza Sul"},

    # ============================================================
    #  LUNDA NORTE & LUNDA SUL
    # ============================================================
    {"nome": "Museu do Dundo", "lat": -7.3789, "lon": 20.8358, "provincia": "Lunda Norte"},
    {"nome": "Cataratas de Luachimo", "lat": -7.3667, "lon": 20.8500, "provincia": "Lunda Norte"},
    {"nome": "Monumento do Soldado Desconhecido (Saurimo)", "lat": -9.6614, "lon": 20.3914, "provincia": "Lunda Sul"},
    {"nome": "Rio Cassai", "lat": -9.8333, "lon": 21.5000, "provincia": "Lunda Sul"},
    # ============================================================
    #  LUANDA
    # ============================================================
    
    {"nome": "Ilha do Mussulo", "lat": -8.964635, "lon": 13.050462, "provincia": "Luanda"},
    {"nome": "Marginal de Luanda", "lat": -8.810556, "lon": 13.230278, "provincia": "Luanda"},
    {"nome": "Banco Nacional de Angola", "lat": -8.812328, "lon": 13.231268, "provincia": "Luanda"},
    {"nome": "Museu da Moeda", "lat": -8.811394, "lon": 13.231845, "provincia": "Luanda"},
    {"nome": "Miradouro da Lua", "lat": -9.221147, "lon": 13.090001, "provincia": "Luanda"},
    {"nome": "Museu Nacional de Antropologia", "lat": -8.816400, "lon": 13.228200, "provincia": "Luanda"},
    {"nome": "Praia de Cabo Ledo", "lat": -9.673323, "lon": 13.199464, "provincia": "Luanda"},
    {"nome": "Praia de Sangano", "lat": -9.519391, "lon": 13.262529, "provincia": "Luanda"},
    {"nome": "Parque Nacional da Quiçama", "lat": -9.750007, "lon": 13.583333, "provincia": "Luanda"},
    {"nome": "Rápidos do Kwanza", "lat": -9.3175, "lon": 13.5688, "provincia": "Luanda"},
    {"nome": "Centro Cultural Agostinho Neto", "lat": -8.8142, "lon": 13.2307, "provincia": "Luanda"},
    {"nome": "Palácio de Ferro", "lat": -8.8159, "lon": 13.2274, "provincia": "Luanda"},
    {"nome": "Largo da Independência", "lat": -8.8315, "lon": 13.2355, "provincia": "Luanda"},
    {"nome": "Museu de História Natural", "lat": -8.8197, "lon": 13.2268, "provincia": "Luanda"},
    {"nome": "Museu Nacional da Escravatura", "lat": -8.9897, "lon": 13.1232, "provincia": "Luanda"},
    {"nome": "Igreja de Nossa Senhora dos Remédios", "lat": -8.8126, "lon": 13.2302, "provincia": "Luanda"},
    {"nome": "Igreja de Jesus", "lat": -8.8145, "lon": 13.2291, "provincia": "Luanda"},
    {"nome": "Mausoléu de Agostinho Neto", "lat": -8.8115, "lon": 13.2405, "provincia": "Luanda"},
    {"nome": "Fortaleza de São Miguel", "lat": -8.808343, "lon": 13.223444, "provincia": "Luanda"},

    # ============================================================
    #  BENGUELA
    # ============================================================
    {"nome": "Baía Azul", "lat": -12.627978, "lon": 13.234185, "provincia": "Benguela"},
    {"nome": "Praia da Baía Farta", "lat": -12.6075, "lon": 13.2008, "provincia": "Benguela"},
    {"nome": "Praia Morena", "lat": -12.5765, "lon": 13.4093, "provincia": "Benguela"},
    {"nome": "Praia da Caóta", "lat": -12.7238, "lon": 13.1363, "provincia": "Benguela"},
    {"nome": "Praia da Catumbela", "lat": -12.4468, "lon": 13.5245, "provincia": "Benguela"},
    {"nome": "Praia do Lobito", "lat": -12.3553, "lon": 13.5592, "provincia": "Benguela"},
    {"nome": "A Restinga (Lobito)", "lat": -12.3333, "lon": 13.5708, "provincia": "Benguela"},
    {"nome": "Praia das Palmeiras", "lat": -12.595, "lon": 13.388, "provincia": "Benguela"},
    {"nome": "Cavaco (zona ribeirinha)", "lat": -12.6105, "lon": 13.356, "provincia": "Benguela"},
    {"nome": "Igreja da Nossa Senhora do Pópulo", "lat": -12.5833, "lon": 13.405, "provincia": "Benguela"},
    {"nome": "Mercado Municipal de Benguela", "lat": -12.5815, "lon": 13.4074, "provincia": "Benguela"},
    {"nome": "Ruínas do Forte de São Pedro", "lat": -12.45, "lon": 13.5333, "provincia": "Benguela"},
    {"nome": "Fazenda Camumbila (ecoturismo)", "lat": -12.793, "lon": 13.844, "provincia": "Benguela"},
    {"nome": "Ponta do Cuio", "lat": -12.693, "lon": 13.161, "provincia": "Benguela"},
    # ============================================================
    #  BENGO
    # ============================================================
    {"nome": "Praia de Caxito", "lat": -8.5786, "lon": 13.6644, "provincia": "Bengo"},
    {"nome": "Lagoa do Panguila", "lat": -8.7611, "lon": 13.4333, "provincia": "Bengo"},
    {"nome": "Foz do Rio Dande", "lat": -8.4986, "lon": 13.3833, "provincia": "Bengo"},
    {"nome": "Parque Nacional da Kissama (Lado Bengo)", "lat": -9.1986, "lon": 13.4912, "provincia": "Bengo"},

    # ============================================================
    #  HUÍLA
    # ============================================================
    {"nome": "Cristo Rei (Lubango)", "lat": -14.940039, "lon": 13.511860, "provincia": "Huíla"}, 
    {"nome": "Covil do Galo", "lat": -14.93, "lon": 13.483, "provincia": "Huíla"},
    {"nome": "Cascata da Huíla", "lat": -14.896, "lon": 13.628, "provincia": "Huíla"},
     {"nome": "Fenda da Tundavala", "lat": -14.817465, "lon": 13.381539, "provincia": "Huíla"},
    {"nome": "Jardim do Ulundo", "lat": -14.885, "lon": 13.541, "provincia": "Huíla"},
    {"nome": "Miradouro da Chela", "lat": -15.1, "lon": 13.25, "provincia": "Huíla"},
    {"nome": "Monumento ao Milenário", "lat": -14.918, "lon": 13.497, "provincia": "Huíla"},
    {"nome": "Planalto da Huíla", "lat": -14.75, "lon": 14.5, "provincia": "Huíla"}, 
    {"nome": "Parque Nacional da Bicuar", "lat": -15.1667, "lon": 15.0, "provincia": "Huíla"}, 
    {"nome": "Tundavala Eco Resort", "lat": -14.814, "lon": 13.385, "provincia": "Huíla"},
    {"nome": "Lagoa da Lua (Huíla)", "lat": -14.771, "lon": 13.558, "provincia": "Huíla"},
    {"nome": "Ponte da Neves", "lat": -15.021, "lon": 13.351, "provincia": "Huíla"},
    {"nome": "Serra da Leba", "lat": -15.078800, "lon": 13.239700, "provincia": "Huíla"},

    # ============================================================
    #  NAMIBE
    # ============================================================
    {"nome": "Deserto do Namibe", "lat": -15.766760, "lon": 12.399914, "provincia": "Namibe"}, 
    {"nome": "Arco do Carvalhão", "lat": -15.75, "lon": 11.8333, "provincia": "Namibe"},
    {"nome": "Praia das Miragens", "lat": -15.183, "lon": 12.15, "provincia": "Namibe"},
    {"nome": "Praia da Amélia", "lat": -15.197, "lon": 12.148, "provincia": "Namibe"},
    {"nome": "Oásis do Arco", "lat": -15.823, "lon": 11.9, "provincia": "Namibe"},
    {"nome": "Parque Nacional do Iona", "lat": -16.6667, "lon": 12.3333, "provincia": "Namibe"},
    {"nome": "Dunas do Namibe", "lat": -15.2, "lon": 12.1, "provincia": "Namibe"},
    {"nome": "Tômbua", "lat": -15.8, "lon": 11.85, "provincia": "Namibe"},
    {"nome": "Resort do Pululukwa", "lat": -14.992, "lon": 13.507, "provincia": "Huíla"}, 
    {"nome": "Monte Negro (Namibe)", "lat": -17.216, "lon": 11.766, "provincia": "Cunene"}, 
    {"nome": "Lagoa dos Flamingos", "lat": -15.79, "lon": 11.89, "provincia": "Namibe"},
    
    
    # ============================================================
    #  MALANJE
    # ============================================================
    {"nome": "Pedras Negras de Pungo Andongo", "lat": -9.7408, "lon": 15.5975, "provincia": "Malanje"},
    {"nome": "Quedas do Lucala", "lat": -9.075025, "lon": 16.001131, "provincia": "Cuanza Norte"}, 
    {"nome": "Parque Nacional de Cangandala", "lat": -9.75, "lon": 16.6667, "provincia": "Malanje"},
    {"nome": "Serra de Palanca", "lat": -9.9, "lon": 16.5, "provincia": "Malanje"},
    
    # ============================================================
    #  HUAMBO
    # ============================================================
    {"nome": "Morro do Môco", "lat": -12.466, "lon": 15.176, "provincia": "Huambo"},
    {"nome": "Cristo Rei (Huambo)", "lat": -12.775, "lon": 15.733, "provincia": "Huambo"},
    {"nome": "Jardim Botânico do Huambo", "lat": -12.78, "lon": 15.73, "provincia": "Huambo"},
    {"nome": "Grutas da Ema", "lat": -12.8, "lon": 15.25, "provincia": "Huambo"},
    {"nome": "Lagoa do Cuima", "lat": -13.06, "lon": 15.39, "provincia": "Huambo"},
    {"nome": "Rio Cunene (Huambo)", "lat": -13.1, "lon": 15.1, "provincia": "Huambo"}, 
    {"nome": "Serra do Gungo", "lat": -12.6, "lon": 15.9, "provincia": "Huambo"},
    {"nome": "Cascatas da Bomba Alta", "lat": -12.75, "lon": 15.78, "provincia": "Huambo"},
    {"nome": "Museu Regional do Huambo", "lat": -12.776, "lon": 15.739, "provincia": "Huambo"},

    # ============================================================
    #  BIÉ
    # ============================================================
    {"nome": "Nascente do Rio Kwanza", "lat": -11.916, "lon": 17.416, "provincia": "Bié"},
    {"nome": "Rio Cuquema", "lat": -12.5, "lon": 17.0, "provincia": "Bié"},
    {"nome": "Serra da Canda", "lat": -12.2, "lon": 17.8, "provincia": "Bié"},
    {"nome": "Museu Regional do Bié", "lat": -12.383, "lon": 16.933, "provincia": "Bié"},
    {"nome": "Caverna de Nhia", "lat": -12.6, "lon": 16.7, "provincia": "Bié"},
    {"nome": "Jardim da Pouca-Vergonha (Kuito)", "lat": -12.3833, "lon": 16.9333, "provincia": "Bié"},
    {"nome": "Morro da Chimbango", "lat": -12.4333, "lon": 16.8500, "provincia": "Bié"},
    
    # ============================================================
    #  MOXICO
    # ============================================================
    {"nome": "Lago Dilolo", "lat": -11.516, "lon": 20.1, "provincia": "Moxico"},
    {"nome": "Rio Luena", "lat": -11.783, "lon": 19.916, "provincia": "Moxico"},
    {"nome": "Serra do Moxico", "lat": -11.5, "lon": 19.0, "provincia": "Moxico"},
    {"nome": "Nascente do Rio Zambeze", "lat": -11.185, "lon": 24.301, "provincia": "Moxico"},
    {"nome": "Quedas do Rio Chiumbe", "lat": -11.021380, "lon": 20.203180, "provincia": "Moxico"},
    {"nome": "Parque Nacional de Cameia", "lat": -11.883338, "lon": 21.666681, "provincia": "Moxico"},

    # ============================================================
    #  UÍGE
    # ============================================================
    {"nome": "Quedas do Lulovo", "lat": -7.7, "lon": 15.3, "provincia": "Uíge"},
    {"nome": "Serra do Pingano", "lat": -7.1, "lon": 14.8, "provincia": "Uíge"},
    {"nome": "Rio Loge", "lat": -7.83, "lon": 13.1, "provincia": "Uíge"},
    {"nome": "Floresta do Kitombe", "lat": -7.4, "lon": 15.5, "provincia": "Uíge"},
    {"nome": "Grutas do Nzenzo", "lat": -7.520192, "lon": 14.565103, "provincia": "Uíge"},
     {"nome": "Igreja de São José (Uíge)", "lat": -7.6086, "lon": 15.0614, "provincia": "Uíge"},
    {"nome": "Mercado do Songo", "lat": -7.1167, "lon": 15.0500, "provincia": "Uíge"},

    # ============================================================
    #  CABINDA
    # ============================================================
    {"nome": "Floresta do Mayombe", "lat": -4.833, "lon": 12.5, "provincia": "Cabinda"}, 
    {"nome": "Praia de Malembo", "lat": -5.366, "lon": 12.183, "provincia": "Cabinda"},
    {"nome": "Cataratas de Lândana", "lat": -5.166, "lon": 12.183, "provincia": "Cabinda"},
    {"nome": "Igreja de Nossa Senhora da Conceição (Cabinda)", "lat": -5.55, "lon": 12.2, "provincia": "Cabinda"},
    {"nome": "Cemitério dos Reis de Cabinda", "lat": -5.5583, "lon": 12.1931, "provincia": "Cabinda"},
    {"nome": "Lagoa de Sassa-Zau", "lat": -4.8986, "lon": 12.1167, "provincia": "Cabinda"},

    # ============================================================
    #  Cuando cubango
    # ============================================================
    {"nome": "Bacia do Cubango-Okavango", "lat": -17.5, "lon": 22.5, "provincia": "Cuando Cubango"}, 
    {"nome": "Cuito Cuanavale (Memorial da Batalha)", "lat": -15.1667, "lon": 19.1667, "provincia": "Cuando Cubango"},
    {"nome": "Parque Nacional de Luengue-Luiana", "lat": -16.2500, "lon": 21.5000, "provincia": "Cuando Cubango"},
    {"nome": "Rio Cuando", "lat": -17.8167, "lon": 23.3333, "provincia": "Cuando Cubango"},
    
    # ============================================================
    #ZAIRE
    # ============================================================
     {"nome": "Museu Kulumbimbi", "lat": -6.264389, "lon": 14.245581, "provincia": "Zaire"},
      {"nome": "Foz do Rio Congo", "lat": -6.0767, "lon": 12.3551, "provincia": "Zaire"},
    {"nome": "Praia dos Pescadores (Soyo)", "lat": -6.1364, "lon": 12.3686, "provincia": "Zaire"},
    {"nome": "Ponta do Padrão", "lat": -6.0786, "lon": 12.3167, "provincia": "Zaire"}
]

BUFFER_RADIUS = 5000  # 5 km
dfs = []

# === Tags OSM — Turismo + Infraestrutura ===
TAGS = {
    "tourism": True,
    "amenity": True,
    "leisure": True,
    "natural": True,
    "landuse": True,
    "highway": True,
    "waterway": True,
    "power": True,
    "man_made": True,
    "railway": True,
    "building": True
}


def coletar_pois(lat, lon, tags, buffer_m):
    """Retorna os POIs dentro do raio (buffer_m) ao redor das coordenadas."""
    try:
        gdf_ponto = gpd.GeoDataFrame([["ponto", Point(lon, lat)]],
                                     columns=["nome", "geometry"],
                                     crs="EPSG:4326")
        area_busca = gdf_ponto.to_crs(3857).buffer(buffer_m).to_crs(4326).iloc[0]

        pois = ox.features_from_polygon(area_busca, tags).reset_index()
        if pois.empty:
            return None

        # Garantir colunas
        colunas_necessarias = [
            "osm_id", "name", "tourism", "amenity", "leisure",
            "natural", "landuse", "highway", "waterway",
            "power", "man_made", "railway", "building", "geometry"
        ]
        for c in colunas_necessarias:
            if c not in pois.columns:
                pois[c] = None
        return pois[colunas_necessarias]
    except Exception as e:
        print(f" Erro na coleta: {e}")
        return None


# === Loop pelos locais ===
for item in locais_turisticos:
    nome = item["nome"]
    lat = item["lat"]
    lon = item["lon"]
    prov = item["provincia"]

    print(f"\n Coletando POIs para {nome} ({prov})...")

    pois = coletar_pois(lat, lon, TAGS, BUFFER_RADIUS)

    if pois is not None:
        # Adicionar metadados
        pois["ponto_turistico"] = nome
        pois["provincia"] = prov
        pois["fonte_dados"] = "OpenStreetMap"
        pois["latitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).y
        pois["longitude"] = pois.to_crs(3857).geometry.centroid.to_crs(4326).x

        # Salvar individualmente
        output_path = DATA_DIR / f"pois_{nome.replace(' ', '_').lower()}.geojson"
        pois.to_file(output_path, driver="GeoJSON")
        print(f"    Salvo: {output_path}")

        dfs.append(pois)
    else:
        print(f" Nenhum POI encontrado em {nome}.")

    time.sleep(1.5)  # respeitar limite de requisições

# === Combinar tudo ===
if dfs:
    pois_final = gpd.GeoDataFrame(pd.concat(dfs, ignore_index=True), crs="EPSG:4326")
    output_final = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
    pois_final.to_file(output_final, driver="GeoJSON")
    print(f"\n Arquivo combinado salvo em: {output_final}")
else:
    print("\n Nenhum POI foi coletado.")


KeyboardInterrupt: 

In [1]:
# Normalizar 

# ==============================================================
# NORMALIZAÇÃO DOS DADOS GEOFÍSICOS E DE INFRAESTRUTURA
# ==============================================================

import geopandas as gpd
from pathlib import Path

DATA_DIR = Path("../data/geoespacial")
input_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson"
output_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson"

# ===  Carregar dados brutos ===
pois = gpd.read_file(input_path)

# ===  Limpeza básica ===
if "osm_id" in pois.columns:
    pois = pois.drop_duplicates(subset=["osm_id"], keep="first")
else:
    pois = pois.drop_duplicates(subset=["name", "geometry"], keep="first")

pois = pois[pois.is_valid]
pois = pois.dropna(subset=["geometry"])

# ===  Garantir colunas críticas ===
for col in ["name", "tourism", "amenity", "leisure", "natural", "landuse", 
            "highway", "power", "latitude", "longitude"]:
    if col not in pois.columns:
        pois[col] = None

# ===  Correção e normalização ===
pois["name"] = pois["name"].fillna("Desconhecido").astype(str)

map_tourism = {
    "guest_house": "accommodation",
    "hotel": "accommodation",
    "motel": "accommodation",
    "museum": "culture",
    "attraction": "attraction"
}
map_amenity = {
    "restaurant": "food_service",
    "fast_food": "food_service",
    "hospital": "health",
    "school": "education"
}

pois["tourism"] = pois["tourism"].replace(map_tourism)
pois["amenity"] = pois["amenity"].replace(map_amenity)

# ===  Variáveis derivadas ===
pois["is_touristic"] = pois["tourism"].notna().astype(int)
pois["is_infrastructure"] = pois[["amenity", "highway", "power"]].notna().any(axis=1).astype(int)

# ===  Salvar resultado ===
pois.to_file(output_path, driver="GeoJSON")
print(f" Dados normalizados salvos em: {output_path}")




 Dados normalizados salvos em: ..\data\geoespacial\pontos_turisticos_angola_normalizado.geojson


In [ ]:
# ==============================================================
# NORMALIZAÇÃO DOS DADOS DE MOBILIDADE E DE INFRAESTRUTURA
# ==============================================================

import pandas as pd
import geopandas as gpd
from pathlib import Path

# === Diretório e arquivo ===
DATA_DIR = Path("../data/climatic-environmental")
DATA_DIR.mkdir(parents=True, exist_ok=True)
input_path = DATA_DIR / "mobilidade_infra.csv"

# === Carregar ===
df = pd.read_csv(input_path)

# === Normalizar nomes de colunas ===
df.columns = df.columns.str.strip().str.lower().str.replace(r'\s+', '_', regex=True)

# === Preenchimento básico ===
num_cols = df.select_dtypes(include=["number"]).columns.tolist()
for c in num_cols:
    df[c] = df[c].fillna(df[c].median())

cat_cols = df.select_dtypes(include=["object", "category"]).columns.tolist()
for c in cat_cols:
    df[c] = df[c].fillna("desconhecido")

# === Limites plausíveis ===
if "velocidade_media_kmh" in df.columns:
    df["velocidade_media_kmh"] = df["velocidade_media_kmh"].clip(lower=0, upper=200)

if "capacidade" in df.columns:
    df["capacidade"] = df["capacidade"].clip(lower=0)

if "fluxo_diario" in df.columns:
    df["fluxo_diario"] = df["fluxo_diario"].clip(lower=0)

# === Datas ===
for date_col in ["date", "data", "timestamp"]:
    if date_col in df.columns:
        df[date_col] = pd.to_datetime(df[date_col], errors="coerce")

# === Normalizar categorias de infraestrutura ===
infra_col_candidates = [c for c in df.columns if c in ("tipo", "tipo_infra", "infra_type", "category")]
if infra_col_candidates:
    infra_col = infra_col_candidates[0]
    df[infra_col] = df[infra_col].astype(str).str.lower()
    df["is_road"] = df[infra_col].str.contains(r"road|highway|estrada", na=False).astype(int)
    df["is_rail"] = df[infra_col].str.contains(r"rail|ferrovia", na=False).astype(int)
    df["is_port"] = df[infra_col].str.contains(r"port|porto", na=False).astype(int)
    df["is_airport"] = df[infra_col].str.contains(r"airport|aeroporto", na=False).astype(int)

# === Coordenadas -> GeoDataFrame ===
lat_candidates = [c for c in df.columns if c in ("latitude", "lat", "y")]
lon_candidates = [c for c in df.columns if c in ("longitude", "lon", "lng", "x")]

output_csv = DATA_DIR / "mobilidade_infra_normalizado.csv"
output_geo = DATA_DIR / "mobilidade_infra_normalizado.geojson"

if lat_candidates and lon_candidates:
    lat_col = lat_candidates[0]
    lon_col = lon_candidates[0]
    df = df.dropna(subset=[lat_col, lon_col])
    gdf = gpd.GeoDataFrame(
        df.copy(),
        geometry=gpd.points_from_xy(df[lon_col].astype(float), df[lat_col].astype(float)),
        crs="EPSG:4326"
    )
    gdf.to_file(output_geo, driver="GeoJSON")
    df.to_csv(output_csv, index=False)
    print(f"Dados de mobilidade normalizados salvos em:\n   - {output_csv}\n   - {output_geo}")
else:
    df.to_csv(output_csv, index=False)
    print(f"Dados de mobilidade normalizados salvos em: {output_csv}")



✅ Dados de mobilidade normalizados salvos em: ..\data\climatic-environmental\mobilidade_infra_normalizado.csv


In [17]:
import pandas as pd
from pathlib import Path

# ==============================================================
# NORMALIZAÇÃO DOS DADOS CLIMATICOS-AMBIENTAIS

# ==============================================================
# === Diretório e arquivos ===
DATA_DIR = Path("../data/climatic-environmental")
DATA_DIR.mkdir(parents=True, exist_ok=True)

input_clima = DATA_DIR / "exemplo.csv"
df_clima = pd.read_csv(input_clima)

# === Normalização dos dados ===
# Temperatura média anual (°C)
df_clima['temp_med_anual'] = df_clima['temp_med_anual'].clip(20, 35)

# Precipitação anual (mm)
df_clima['precipitacao_anual'] = df_clima['precipitacao_anual'].clip(0, 2000)

# Índices de vegetação e água
for col in ['NDVI', 'EVI', 'NDWI']:
    df_clima[col] = df_clima[col].clip(-1, 1)

# Códigos LULC (Land Use Land Cover)
df_clima['LULC_codigo'] = df_clima['LULC_codigo'].astype('category')

# Altitude (m)
df_clima['altitude'] = df_clima['altitude'].clip(0, 2500)

# === Salvar dados normalizados ===
output_path = DATA_DIR / "clima_lulc_normalizado.csv"
df_clima.to_csv(output_path, index=False)
print(f" Dados climáticos-ambientais normalizados salvos em: {output_path}")

 Dados climáticos-ambientais normalizados salvos em: ..\data\climatic-environmental\clima_lulc_normalizado.csv


In [7]:
import pandas as pd
from pathlib import Path

# ==============================================================
# NORMALIZAÇÃO DOS DADOS ECONÔMICOS-SOCIAIS
# ==============================================================
# === Diretório e arquivos ===
DATA_DIR = Path("../data/economicsocial")
DATA_DIR.mkdir(parents=True, exist_ok=True)

input_eco = DATA_DIR / "dados_socioeconomicos.csv"
df_eco = pd.read_csv(input_eco)

# === Normalização dos dados ===
# PIB per capita (USD)
df_eco['pib_per_capita'] = df_eco['pib_per_capita'].clip(500, 5000)

# População
df_eco['populacao'] = df_eco['populacao'].clip(1000, 1000000)

# Índice de desenvolvimento humano
df_eco['idh'] = df_eco['idh'].clip(0, 1)

# Densidade populacional (hab/km²)
df_eco['densidade_pop'] = df_eco['densidade_pop'].clip(0, 1000)

# Taxa de urbanização (%)
df_eco['taxa_urbanizacao'] = df_eco['taxa_urbanizacao'].clip(0, 100)

# === Salvar dados normalizados ===
output_path = DATA_DIR / "economicsocial_normalizado.csv"
df_eco.to_csv(output_path, index=False)
print(f" Dados econômicos-sociais normalizados salvos em: {output_path}")

 Dados econômicos-sociais normalizados salvos em: ..\data\economicsocial\economicsocial_normalizado.csv


In [2]:
# Gerar variáveis espaciais 
# ============================================================== 
# GERAR VARIÁVEIS ESPACIAIS
# ==============================================================

import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from pathlib import Path
import numpy as np

DATA_DIR = Path("../data/geoespacial")
input_path = DATA_DIR / "pontos_turisticos_angola_normalizado.geojson"
output_path = DATA_DIR / "pontos_turisticos_angola_raw.geojson"

# === Carregar dados normalizados ===
pois = gpd.read_file(input_path)

# === Converter para projeção métrica (metros) para cálculos espaciais ===
pois = pois.to_crs(epsg=3857)

# === Exemplo 1: Calcular centroid de cada ponto turístico (se houver polígonos) ===
# (No caso de pontos simples, é só o próprio ponto)
pois['centroid_x'] = pois.geometry.centroid.x
pois['centroid_y'] = pois.geometry.centroid.y

# === Exemplo 2: Distância até o ponto turístico mais próximo ===
# Criar uma função para calcular a menor distância entre cada ponto
def nearest_distance(point, others):
    # Retorna a menor distância (em metros) até outro ponto
    distances = others.geometry.distance(point)
    distances = distances[distances > 0]  # ignora distância zero (o próprio ponto)
    if len(distances) > 0:
        return distances.min()
    else:
        return np.nan

pois['dist_to_nearest_poi'] = pois.geometry.apply(lambda p: nearest_distance(p, pois))

# === Exemplo 3: Buffer / contagem de POIs próximos (densidade) ===
BUFFER_RADIUS = 1000  # metros
pois['pois_within_1km'] = pois.geometry.apply(lambda g: pois[pois.geometry.within(g.buffer(BUFFER_RADIUS))].shape[0] - 1) 
# -1 para excluir o próprio ponto

# === Exemplo 4: Flag de proximidade a categorias específicas ===
# Por exemplo, se existe hospital ou escola em 1km
def has_category_nearby(point, category_col, category_value, radius=1000):
    buffer = point.buffer(radius)
    subset = pois[(pois[category_col] == category_value) & (pois.geometry.within(buffer))]
    return int(len(subset) > 0)

pois['hospital_nearby'] = pois.geometry.apply(lambda g: has_category_nearby(g, 'amenity', 'hospital'))
pois['school_nearby'] = pois.geometry.apply(lambda g: has_category_nearby(g, 'amenity', 'school'))

# === Salvar dataset com variáveis espaciais ===
pois = pois.to_crs(epsg=4326)  # voltar para lat/lon
pois.to_file(output_path, driver="GeoJSON")
print(f" Variáveis espaciais geradas e salvas em: {output_path}")


 Variáveis espaciais geradas e salvas em: ..\data\geoespacial\pontos_turisticos_angola_raw.geojson


In [5]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from pathlib import Path
import re

# ============================================================
# 🔹 FUNÇÕES AUXILIARES
# ============================================================

def normalizar(texto):
    """Remove acentos, espaços extras e converte em minúsculas."""
    if pd.isna(texto):
        return ""
    texto = str(texto).strip().lower()
    texto = re.sub(r"[áàãâä]", "a", texto)
    texto = re.sub(r"[éèêë]", "e", texto)
    texto = re.sub(r"[íìîï]", "i", texto)
    texto = re.sub(r"[óòõôö]", "o", texto)
    texto = re.sub(r"[úùûü]", "u", texto)
    texto = re.sub(r"ç", "c", texto)
    texto = re.sub(r"[^a-z0-9 ]", "", texto)
    return texto


def carregar_csv(caminho):
    """Lê CSV corrigindo encoding e normalizando nomes de colunas."""
    df = pd.read_csv(caminho, encoding="utf-8", on_bad_lines="skip")
    df.columns = [normalizar(c) for c in df.columns]
    return df


def adicionar_chave(df, colunas_possiveis):
    """Adiciona coluna 'key' com nome normalizado."""
    for col in df.columns:
        if any(col.startswith(c) for c in colunas_possiveis):
            df["key"] = df[col].astype(str).apply(normalizar)
            return df
    raise ValueError(f"❌ Nenhuma coluna de nome encontrada. Colunas disponíveis: {list(df.columns)}")


# ============================================================
# 🔹 CAMINHOS DOS DADOS
# ============================================================

BASE_DIR = Path("../data")
PATH_GEO_NORM = BASE_DIR / "geoespacial/pontos_turisticos_angola_normalizado.geojson"
PATH_GEO_RAW = BASE_DIR / "geoespacial/pontos_turisticos_angola_raw.geojson"
PATH_CLIMA = BASE_DIR / "climatic-environmental/clima_lulc_normalizado.csv"
PATH_MOB = BASE_DIR / "climatic-environmental/mobilidade_infra_normalizado.csv"
PATH_SOCIO = BASE_DIR / "economicsocial/economicsocial_normalizado.csv"

# ============================================================
# 🔹 CARREGAR CAMADAS
# ============================================================

print("🔹 Carregando camadas de dados...")

gdf_norm = gpd.read_file(PATH_GEO_NORM)
gdf_raw = gpd.read_file(PATH_GEO_RAW)
df_clima = carregar_csv(PATH_CLIMA)
df_mob = carregar_csv(PATH_MOB)
df_socio = carregar_csv(PATH_SOCIO)

# ============================================================
# 🔹 ADICIONAR CHAVES NORMALIZADAS
# ============================================================

print("🔹 Normalizando nomes e criando chaves...")

gdf_norm = adicionar_chave(gdf_norm, ["ponto_turistico", "name", "nome"])
gdf_raw = adicionar_chave(gdf_raw, ["ponto_turistico", "name", "nome"])
df_clima = adicionar_chave(df_clima, ["nome_ponto_turistico", "nome"])
df_mob = adicionar_chave(df_mob, ["nome"])
df_socio = adicionar_chave(df_socio, ["nome"])

# ============================================================
# 🔹 JUNÇÃO SEMÂNTICA — CLIMA + SOCIO + MOBILIDADE
# ============================================================

print("🔹 Realizando junção semântica (clima + socioeconômico + mobilidade)...")

df_semantico = df_clima.merge(df_socio, on="key", how="outer", suffixes=("_clima", "_socio"))
df_semantico = df_semantico.merge(df_mob, on="key", how="outer", suffixes=("", "_mob"))

# ============================================================
# 🔹 JUNÇÃO ESPACIAL — COM GEOESPACIAL NORMALIZADO E RAW
# ============================================================

print("🔹 Realizando junção espacial com dados geográficos...")

# Converter para GeoDataFrame com base em coordenadas (lat/lon)
if "longitude" in gdf_norm.columns and "latitude" in gdf_norm.columns:
    gdf_norm["geometry"] = gdf_norm.apply(lambda x: Point(x["longitude"], x["latitude"]), axis=1)
gdf_norm = gpd.GeoDataFrame(gdf_norm, geometry="geometry", crs="EPSG:4326")

# Merge final com base na chave
gdf_final = gdf_norm.merge(df_semantico, on="key", how="left")
gdf_final = gdf_final.merge(gdf_raw.drop(columns=["geometry"]), on="key", how="left", suffixes=("", "_raw"))

# ============================================================
# 🔹 EXPORTAR RESULTADOS
# ============================================================

OUTPUT_DIR = BASE_DIR / "integrado"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

csv_out = OUTPUT_DIR / "dataset_integrado.csv"
geojson_out = OUTPUT_DIR / "dataset_integrado.geojson"

print(f" Salvando arquivos em: {OUTPUT_DIR}")
gdf_final.to_csv(csv_out, index=False, encoding="utf-8")
gdf_final.to_file(geojson_out, driver="GeoJSON")

print(" Integração concluída com sucesso!")
print(f" Arquivos gerados:\n - {csv_out.name}\n - {geojson_out.name}")


🔹 Carregando camadas de dados...
🔹 Normalizando nomes e criando chaves...
🔹 Realizando junção semântica (clima + socioeconômico + mobilidade)...
🔹 Realizando junção espacial com dados geográficos...
 Salvando arquivos em: ..\data\integrado
 Integração concluída com sucesso!
 Arquivos gerados:
 - dataset_integrado.csv
 - dataset_integrado.geojson


In [ ]:
import pandas as pd
from pathlib import Path

# =====================================================
#  Gerar model_input.csv consolidado e pronto p/ ML
# =====================================================

# Caminhos dos arquivos
DATA_CLIMA = Path("../data/climatic-environmental/clima_lulc_normalizado.csv")
DATA_MOB = Path("../data/climatic-environmental/mobilidade_infra_normalizado.csv")
DATA_SOCIO = Path("../data/economicsocial/economicsocial_normalizado.csv")

# === 1. Carregar dados ===
df_clima = pd.read_csv(DATA_CLIMA)
df_mob = pd.read_csv(DATA_MOB)
df_socio = pd.read_csv(DATA_SOCIO)

# === 2. Normalizar nomes e chaves ===
def normalize_name(n):
    return (
        str(n)
        .strip()
        .lower()
        .replace("ã", "a")
        .replace("ç", "c")
        .replace("í", "i")
        .replace("é", "e")
        .replace("ú", "u")
        .replace("ó", "o")
    )

for df in [df_clima, df_mob, df_socio]:
    df.columns = df.columns.str.strip().str.lower()
    if "nome_ponto_turistico" in df.columns:
        df["key"] = df["nome_ponto_turistico"].apply(normalize_name)
    elif "nome" in df.columns:
        df["key"] = df["nome"].apply(normalize_name)

# === 3. Merge dos datasets ===
df_merge = (
    df_clima.merge(df_socio, on="key", suffixes=("_clima", "_socio"))
    .merge(df_mob, on="key", suffixes=("", "_mob"))
)

# === 4. Selecionar colunas finais ===
colunas_finais = [
    "nome_ponto_turistico_clima",
    "provincia_clima",
    "lat_clima",
    "lon_clima",
    "temp_med_anual",
    "precipitacao_anual",
    "ndvi",
    "evi",
    "ndwi",
    "altitude",
    "populacao",
    "densidade_pop",
    "pib_per_capita",
    "idh",
    "taxa_urbanizacao",
    "emprego_turismo",
    "distancia_estrada_principal_km",
    "distancia_cidade_km",
]

# Garantir que as colunas existam
colunas_existentes = [c for c in colunas_finais if c in df_merge.columns]

# === 5. Criar dataframe final ===
df_final = df_merge[colunas_existentes].copy()

# === 6. Salvar CSV final ===
output_path = Path("../data/model_inputs/model_input.csv")
df_final.to_csv(output_path, index=False)

print(f" Model Input salvo em: {output_path}")
print(df_final.head())

✅ Model Input salvo em: ..\data\model_input.csv
  nome_ponto_turistico_clima provincia_clima  lat_clima  lon_clima  \
0        Quedas de Calandula         Malanje  -9.075025  16.001131   
1           Miradouro da Lua          Luanda  -9.221147  13.090001   
2           Museu Kulumbimbi           Zaire  -6.264389  14.245581   
3  Reserva Parcial do Namibe          Namibe -15.766760  12.399914   
4    Fortaleza de São Miguel          Luanda  -8.808343  13.223445   

   temp_med_anual  precipitacao_anual  ndvi   evi  ndwi  altitude  populacao  \
0            28.8               218.9  0.80  0.16  0.02     287.0      95533   
1            24.8               304.1  0.77  0.68  0.08    1148.0      52641   
2            27.8               343.0  0.20  0.46  0.08     683.0      54008   
3            24.3               286.7  0.15  0.60  0.04     362.0     166079   
4            28.1               158.6  0.62  0.47  0.16    1192.0     192436   

   densidade_pop  pib_per_capita    idh  taxa_urba